Import of Packages

In [1]:
import scipy.stats as stats
import heapq
from collections import defaultdict
#from Code.import_data import import_data
from datetime import datetime, timedelta
import pandas as pd

Import of Data

In [2]:

def import_data():
    # @diana and denis, change to your directory
    gtfs_dir = '/Users/paulinaheine/Codes/BackupTravelPlanning/GTFS_OP_2024_obb-2/'

    # Load data
    agency_df = pd.read_csv(gtfs_dir + 'agency.txt')
    stops_df = pd.read_csv(gtfs_dir + 'stops.txt')
    routes_df = pd.read_csv(gtfs_dir + 'routes.txt')
    trips_df = pd.read_csv(gtfs_dir + 'trips.txt')
    stop_times_df = pd.read_csv(gtfs_dir + 'stop_times.txt')
    calendar_df = pd.read_csv(gtfs_dir + 'calendar.txt')
    calendar_dates_df = pd.read_csv(gtfs_dir + 'calendar_dates.txt')
    shapes_df = pd.read_csv(gtfs_dir + 'shapes.txt')
    
    # Return all DataFrames
    return agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df, calendar_dates_df, shapes_df

# Import and display data


Helping Functions

In [3]:
def time_to_minutes(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 60 + minutes + seconds / 60

def minutes_to_time(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return f"{hours:02d}:{minutes:02d}"

def get_weekday(date):
    weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    return weekdays[date.weekday()]

def is_service_available(service_id, start_time_obj, calendar, calendar_dates):
    #date_str = date.strftime("%Y%m%d")
    #date = date.date()
    # Convert date to string in YYYYMMDD format
    date_str = start_time_obj.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][start_time_obj.weekday()]

    # Step 1: Check for exceptions in calendar_dates
    if service_id in calendar_dates["service_id"].values:
        exceptions = calendar_dates[calendar_dates["service_id"] == service_id]
        for _, exception in exceptions.iterrows():
            if exception["date"] == int(date_str):
                if exception["exception_type"] == 2:  # Service is added as an exception
                    return True
                elif exception["exception_type"] == 1:  # Service is removed as an exception
                    return False

    # Step 2: Check for regular service in calendar
    if service_id in calendar["service_id"].values:
        service = calendar[calendar["service_id"] == service_id]
        # Check if date is within start_date and end_date
        if int(service["start_date"].iloc[0]) <= int(date_str) <= int(service["end_date"].iloc[0]):
            # Check if the service operates on this weekday
            is_available = (service[weekday].iloc[0] == 1)
            if is_available == 1:
                return True
    return False

def prepare_calendar_dates(calendar_dates):
    grouped = calendar_dates.groupby("service_id")
    calendar_dates_dict = {}
    for service_id, group in grouped:
        exceptions = group.to_dict(orient="records")
        calendar_dates_dict[service_id] = exceptions
    return calendar_dates_dict

In [4]:
agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()

Create Graph

In [5]:
def create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj, end_time_obj):
    '''
    funktioniert endlich, if not wurde entfernt weil es mich genervt hat 3-4 min 
    '''

    graph = defaultdict(list)
    stop_id_to_name = stops.set_index("stop_id")["stop_name"].to_dict()
    # Filter for active trips today using is_service
    trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    trip_id_to_route = trips.set_index("trip_id")["route_id"].to_dict()

    calendar_dates_2 = prepare_calendar_dates(calendar_dates)
    stop_times = stop_times.sort_values(by=["trip_id", "stop_sequence"])

    #####
    #Filter
    #####

    stop_times_copy = stop_times.copy()
    # Filter out stops outside the time window
    stop_times_copy["arrival_minutes"] = stop_times_copy["arrival_time"].apply(time_to_minutes)
    stop_times_copy["departure_minutes"] = stop_times_copy["departure_time"].apply(time_to_minutes)

    start_minutes = time_to_minutes(start_time_obj.strftime("%H:%M:%S"))
    end_minutes = time_to_minutes(end_time_obj.strftime("%H:%M:%S"))

    stop_times = stop_times_copy[
        (stop_times_copy["arrival_minutes"] >= start_minutes) &
        (stop_times_copy["departure_minutes"] <= end_minutes)
        ]

    print(f"Rows after time window filter: {len(stop_times)}")


    #trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    stop_times["service_id"] = stop_times["trip_id"].map(trip_id_to_service)

    grouped = stop_times.groupby("trip_id")
    for trip_id, group in grouped:
        service_id = trip_id_to_service.get(trip_id)
        
        if is_service_available(service_id, start_time_obj, calendar, calendar_dates) == True:
    
            stops = group["stop_id"].tolist()
            departures = group["departure_time"].apply(time_to_minutes).tolist()
            arrivals = group["arrival_time"].apply(time_to_minutes).tolist()
    
            for start, end, dep, arr in zip(stops[:-1], stops[1:], departures[:-1], arrivals[1:]):
                travel_time = arr - dep
                if travel_time > 0:
                    graph[stop_id_to_name[start]].append(
                        (stop_id_to_name[end], dep, arr, trip_id_to_route.get(trip_id))
                    )

    return graph


Reliability function

In [62]:
def compute_transfer_probability_with_departure_delay(transfer_time):
    #transfer_window = departure_time - scheduled_arrival
    #if transfer_window < 0:
    #    return 0.1  # No chance of a successful transfer if arrival is after departure
    return stats.gamma.cdf(transfer_time, a=2, scale=3)


A*

(inf, [], 0.0)

A*

MAIN

In [128]:

    backup_routes = []

    last_route = None    

    for i in range(1, len(path_fixed) - 1, 2):  # Alle Umstiegspunkte durchgehen
        transfer_stop, transfer_time = path_fixed[i - 1]
    # Identifiziere alle Umstiegspunkte in der Route
   # for i in range(1, len(path_fixed) - 1, 2):
        
        #adjusted_transfer_time = MIN_TRANSFER_TIME + 15
        current_stop, current_time = path_fixed[i - 1]
        route_id, departure_time, arrival_time = path_fixed[i]
        next_stop, _ = path_fixed[i + 1]

        if route_id != last_route:
            print(current_stop, current_time,route_id, departure_time, arrival_time,next_stop)
        last_route = route_id    

Schattendorf Kirchengasse 870 20-SV5-L-j24-1 883.0 888.0 Loipersbach-Schattendorf Bhf.
Loipersbach-Schattendorf Bhf. 888.0 3-R93-A-j24-1 894.0 898.0 Marz-Rohrbach Bahnhof
Wiener Neustadt Hauptbahnhof 925.0 10-A12-j24-1 930.0 955.0 Wien Meidling
Wien Hauptbahnhof 962.0 10-A3-j24-1 969.0 984.0 Flughafen Wien Bahnhof


# A* 

In [198]:
import heapq
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius der Erde in Kilometern
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Entfernung in Kilometern

def heuristic(current_stop, end_name, stops_df):
    current_stop_row = stops_df[stops_df['stop_name'] == current_stop].iloc[0]
    end_stop_row = stops_df[stops_df['stop_name'] == end_name].iloc[0]

    current_lat = current_stop_row['stop_lat']
    current_lon = current_stop_row['stop_lon']
    end_lat = end_stop_row['stop_lat']
    end_lon = end_stop_row['stop_lon']

    distance_km = haversine(current_lat, current_lon, end_lat, end_lon)

    average_speed = 60  # km/h
    estimated_time_minutes = (distance_km / average_speed) * 60
    return estimated_time_minutes

#heuristic("Wien Handelskai", end_stop_name,  stops)

def a_star_on_speed_fixed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=2, stops_df=None):
    pq = [(start_time_minutes, start_name, [], 1.0, None, 0)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern

    while pq:
        current_time, current_stop, path, reliability, last_route, _ = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]

        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or new_current_time < best_result[0]:  # Wenn das Ziel früher erreicht wird
                best_result = (current_time, path, reliability)


        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time,  # Gesamtbewertung: aktuelle Zeit + Heuristik
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id,
                    h
                ))

    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result_fast
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden


In [199]:
'''
current_time_fast, best_result_fast , reliability_fast = a_star_on_speed_fixed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)
'''

In [200]:
#current_time_fast, best_result_fast , reliability_fast

(984.0,
 [('Schattendorf Kirchengasse', 870),
  ('20-SV5-L-j24-1', 883.0, 888.0),
  ('Loipersbach-Schattendorf Bhf.', 888.0),
  ('3-R93-A-j24-1', 894.0, 898.0),
  ('Marz-Rohrbach Bahnhof', 898.0),
  ('3-R93-A-j24-1', 899.0, 901.0),
  ('Mattersburg Bahnhof', 901.0),
  ('3-R93-A-j24-1', 902.0, 903.0),
  ('Mattersburg Nord Bahnhof', 903.0),
  ('3-R93-A-j24-1', 904.0, 907.0),
  ('Wiesen-Sigleß Bahnhof', 907.0),
  ('3-R93-A-j24-1', 907.0, 911.0),
  ('Bad Sauerbrunn Bahnhof', 911.0),
  ('3-R93-A-j24-1', 913.0, 916.0),
  ('Neudörfl Bahnhof', 916.0),
  ('3-R93-A-j24-1', 917.0, 920.0),
  ('Katzelsdorf Bahnhof', 920.0),
  ('3-R93-A-j24-1', 920.0, 925.0),
  ('Wiener Neustadt Hauptbahnhof', 925.0),
  ('10-A12-j24-1', 930.0, 955.0),
  ('Wien Meidling', 955.0),
  ('10-A1-j24-1', 958.0, 963.0),
  ('Wien Hauptbahnhof', 963.0),
  ('10-A3-j24-1', 969.0, 984.0),
  ('Flughafen Wien Bahnhof', 984.0)],
 np.float64(0.0462739710004139))

In [201]:


def a_star_with_reliability_fixed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=7, stops_df=None):
    pq = [(start_time_minutes, start_name, [], 1.0, None, 0)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern

    while pq:
        current_time, current_stop, path, reliability, last_route, _ = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]

        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or new_reliability > best_result[2]:  # Wenn das Ziel früher erreicht wird
                best_result = (current_time, path, reliability)


        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time <= MIN_TRANSFER_TIME:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time,  # Gesamtbewertung: aktuelle Zeit + Heuristik
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id,
                    h
                ))

    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden


In [202]:
''''
current_time_rel, best_result_rel , reliability_rel = a_star_with_reliability_fixed(graph, start_stop_name, end_stop_name, start_time_minutes, exclude_routes=set(), MIN_TRANSFER_TIME=7)
'''

In [203]:
#current_time_rel, best_result_rel , reliability_rel

(inf, [], 0.0)

In [206]:


def a_star_backups(graph, path_fixed = best_result_fast):

    backup_routes = []
    
    last_route = None
    
    for i in range(1, len(path_fixed) - 1, 2):  # Alle Umstiegspunkte durchgehen
        transfer_stop, transfer_time = path_fixed[i - 1]
        # Identifiziere alle Umstiegspunkte in der Route
        # for i in range(1, len(path_fixed) - 1, 2):
    
        #adjusted_transfer_time = MIN_TRANSFER_TIME + 15
        current_stop, current_time = path_fixed[i - 1]
        route_id, departure_time, arrival_time = path_fixed[i]
        next_stop, _ = path_fixed[i + 1]
    
        if route_id != last_route:
                backup_time, backup_path, backup_reliability = a_star_with_reliability_fixed(
                    graph, transfer_stop, path_fixed[-1][0], transfer_time, stops_df=None,MIN_TRANSFER_TIME=7
                )
                backup_routes.append([backup_time, backup_path, backup_reliability])
        last_route = route_id
        
    return backup_routes

Main

In [209]:

# Hauptpro©ƒgramm
if __name__ == "__main__":

    agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()
    '''
    start_stop_name = "Schattendorf Kirchengasse"
    end_stop_name = "Flughafen Wien Bahnhof"
    start_datetime = "2024-12-12 14:30:00"
    '''
    start_stop_name = "Wien Hauptbahnhof"
    end_stop_name = "Eisenstadt Bahnhof"
    start_datetime = "2024-12-12 14:30:00"

    time_budget = "2:10"
    time_budget_hours, time_budget_minutes = map(int, time_budget.split(":"))
    time_budget_minutes = time_budget_hours * 60 + time_budget_minutes / 60

    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute

    # In ein reines date-Objekt umwandeln
    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    end_time_obj = start_time_obj + timedelta(minutes=time_budget_minutes)
    date_obj = start_time_obj.date()

    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute


    #slow and correct
    print("creating graph")
    graph = create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj, end_time_obj)
    if start_stop_name not in graph or end_stop_name not in graph:
        print("🚨 Ungültige Start- oder Zielhaltestelle!")
        #sys.exit()

    # Haupt-Dijkstra-Lauf
    print("A star")
    current_time_fast, best_result_fast , reliability_fast = a_star_on_speed_fixed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)

    # Backup-Routen berechnen
    print("backups")
    backup_routes = a_star_backups(graph,path_fixed = best_result_fast)

    # Ergebnis anzeigen
    if current_time_fast < float("inf"):
        arrival_time_fixed = minutes_to_time(current_time_fast)
        print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")

        last_route = None
        grouped_routes = []

        for i in range(1, len(best_result_fast) - 1, 2):
            current_stop, current_time = best_result_fast[i - 1]
            route_id, departure_time, arrival_time = best_result_fast[i]
            next_stop, _ = best_result_fast[i + 1]

            if route_id == last_route:
                grouped_routes[-1]["stops"].append((next_stop, arrival_time))
            else:
                grouped_routes.append({
                    "route_id": route_id,
                    "start_stop": current_stop,
                    "departure_time": departure_time,
                    "stops": [(next_stop, arrival_time)]
                })
            last_route = route_id

        for segment in grouped_routes:
            start = segment["start_stop"]
            dep_time = minutes_to_time(segment["departure_time"])
            route = segment["route_id"]
            stops = " → ".join([f"{stop} (Ankunft: {minutes_to_time(arr)})" for stop, arr in segment["stops"]])
            print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

        print(f"\n🎯 Endstation: {end_stop_name} (Ankunft: {arrival_time_fixed})")
        print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability_fast:.2f}\n")

    
    
    
        print("🔄 Backup-Routen:")
        if backup_routes:
            print("\n🔄 Backup-Routen:")
            for stop, path, reliability in backup_routes:
                print(f"  🔁 Backup von {stop}:")
                last_route = None
                first_segment = True

                for segment in path:
                    start, dep_time, end, arr_time, route = segment

                    if last_route == route:
                        print(f" → {end} (Ankunft: {arr_time})", end="")
                    else:
                        if not first_segment:
                            print()
                        print(f"    🚆 {start} (Abfahrt: {dep_time}) → {end} (Ankunft: {arr_time}) mit Linie {route}",
                              end="")
                        first_segment = False

                    last_route = route

                print(f"\n    🔹 Zuverlässigkeit: {reliability:.2f}\n")
        else:
            print("  ❌ Keine Backup-Routen verfügbar.")

        print()
    else:
        print(f"\n⚠️ Keine zuverlässige Route von {start_stop_name} nach {end_stop_name} gefunden.\n")



creating graph
Rows after time window filter: 32584


/var/folders/qw/jbl_sx2j4r114n9k1myblk6c0000gn/T/ipykernel_81996/2638892618.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times["service_id"] = stop_times["trip_id"].map(trip_id_to_service)


djikstra
backups

📍 Optimierte zuverlässigste Route von Flughafen Wien Bahnhof nach Seyring Bahnhof:
  🚆 Flughafen Wien Bahnhof (Abfahrt: 14:33) → Wien Hauptbahnhof (Ankunft: 14:48) mit Linie 10-A1-j24-1
  🚆 Wien Hauptbahnhof (Abfahrt: 14:57) → Wien Quartier Belvedere (Ankunft: 14:58) → Wien Rennweg (Ankunft: 15:01) → Wien Mitte-Landstraße (Ankunft: 15:03) → Wien Praterstern (Ankunft: 15:07) mit Linie 1-S3-W-j24-1
  🚆 Wien Praterstern (Abfahrt: 15:17) → Wien Traisengasse (Ankunft: 15:20) → Wien Handelskai (Ankunft: 15:21) → Wien Floridsdorf (Ankunft: 15:25) → Wien Siemensstraße (Ankunft: 15:29) → Wien Leopoldau (Ankunft: 15:32) → Gerasdorf Bahnhof (Ankunft: 15:37) → Kapellerfeld Bahnhof (Ankunft: 15:39) → Seyring Bahnhof (Ankunft: 15:42) mit Linie 1-S2-W-j24-1

🎯 Endstation: Seyring Bahnhof (Ankunft: 15:42)
🔹 Gesamt-Zuverlässigkeit der Route: 0.68

🔄 Backup-Routen:

🔄 Backup-Routen:
  🔁 Backup von 981.0:


ValueError: not enough values to unpack (expected 5, got 2)

In [256]:
for i in range(len(backup_routes)):
    print("Backup")
    for j in range(len(backup_routes[i])):
        print(backup_routes[i][j])

Backup
981.0
[('Flughafen Wien Bahnhof', 870), ('1-S7-W-j24-1', 885.0, 890.0), ('Fischamend Bahnhof', 890.0), ('1-S7-W-j24-1', 911.0, 915.0), ('Flughafen Wien Bahnhof', 915.0), ('1-S7-W-j24-1', 919.0, 923.0), ('Mannswörth Bahnhof', 923.0), ('1-S7-W-j24-1', 923.0, 925.0), ('Schwechat Bahnhof', 925.0), ('1-S7-W-j24-1', 926.0, 928.0), ('Wien Kaiserebersdorf', 928.0), ('1-S7-W-j24-1', 929.0, 932.0), ('Wien Zentralfriedhof', 932.0), ('1-S7-W-j24-1', 932.0, 935.0), ('Wien Geiselbergstraße', 935.0), ('1-S7-W-j24-1', 935.0, 937.0), ('Wien St. Marx', 937.0), ('1-S7-W-j24-1', 938.0, 940.0), ('Wien Rennweg', 940.0), ('1-S2-W-j24-1', 948.0, 950.0), ('Wien Mitte-Landstraße', 950.0), ('1-S2-W-j24-1', 951.0, 954.0), ('Wien Praterstern', 954.0), ('1-S2-W-j24-1', 956.0, 959.0), ('Wien Traisengasse', 959.0), ('1-S2-W-j24-1', 959.0, 960.0), ('Wien Handelskai', 960.0), ('1-S2-W-j24-1', 961.0, 964.0), ('Wien Floridsdorf', 964.0), ('1-S2-W-j24-1', 965.0, 968.0), ('Wien Siemensstraße', 968.0), ('1-S2-W-j24-1

In [213]:
from tabulate import tabulate

def format_route_data(backup_routes):
    # Wandelt die Daten in ein DataFrame um
    result_data = []
    for route in backup_routes:
        reliability = route[2]  # Zuverlässigkeit
        path = route[1]  # Pfadinformation
        result_data.append([reliability, path])

    df = pd.DataFrame(result_data, columns=["Reliability", "Path"])

    # Ausgabe der Tabelle mit tabulate
    table = tabulate(df, headers='keys', tablefmt='pretty', showindex=False)
    return table

# Aufrufen der Funktion
formatted_output = format_route_data(backup_routes)
print(formatted_output)

+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [239]:
for i in range(1, len(backup_routes[2]) - 1, 2):
    brf = backup_routes[2]
    current_stop, current_time = brf[i - 1]
    route_id, departure_time, arrival_time = brf[i]
    next_stop, _ = brf[i + 1]

TypeError: cannot unpack non-iterable float object

In [238]:
brf[i - 1]

942.0

In [247]:
backup_routes


[[[('Flughafen Wien Bahnhof', 870),
   ('1-S7-W-j24-1', 885.0, 890.0),
   ('Fischamend Bahnhof', 890.0),
   ('1-S7-W-j24-1', 911.0, 915.0),
   ('Flughafen Wien Bahnhof', 915.0),
   ('1-S7-W-j24-1', 919.0, 923.0),
   ('Mannswörth Bahnhof', 923.0),
   ('1-S7-W-j24-1', 923.0, 925.0),
   ('Schwechat Bahnhof', 925.0),
   ('1-S7-W-j24-1', 926.0, 928.0),
   ('Wien Kaiserebersdorf', 928.0),
   ('1-S7-W-j24-1', 929.0, 932.0),
   ('Wien Zentralfriedhof', 932.0),
   ('1-S7-W-j24-1', 932.0, 935.0),
   ('Wien Geiselbergstraße', 935.0),
   ('1-S7-W-j24-1', 935.0, 937.0),
   ('Wien St. Marx', 937.0),
   ('1-S7-W-j24-1', 938.0, 940.0),
   ('Wien Rennweg', 940.0),
   ('1-S2-W-j24-1', 948.0, 950.0),
   ('Wien Mitte-Landstraße', 950.0),
   ('1-S2-W-j24-1', 951.0, 954.0),
   ('Wien Praterstern', 954.0),
   ('1-S2-W-j24-1', 956.0, 959.0),
   ('Wien Traisengasse', 959.0),
   ('1-S2-W-j24-1', 959.0, 960.0),
   ('Wien Handelskai', 960.0),
   ('1-S2-W-j24-1', 961.0, 964.0),
   ('Wien Floridsdorf', 964.0),
   (

In [250]:

    
        
print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")
for x in range(len(backup_routes)):
    print("Backup ab X")
    
    last_route = None
    grouped_routes = []
    brf = backup_routes[x]
    brf.pop(0)

    for i in range(1, len(backup_routes[x]) - 1, 2):
        
        current_stop, current_time = brf[i - 1]
        route_id, departure_time, arrival_time = brf[i]
        next_stop, _ = brf[i + 1]

        if route_id == last_route:
            grouped_routes[-1]["stops"].append((next_stop, arrival_time))
        else:
            grouped_routes.append({
                "route_id": route_id,
                "start_stop": current_stop,
                "departure_time": departure_time,
                "stops": [(next_stop, arrival_time)]
            })
        
        last_route = route_id
        
        for segment in grouped_routes:
            start = segment["start_stop"]
            dep_time = minutes_to_time(segment["departure_time"])
            route = segment["route_id"]
            stops = " → ".join([f"{stop} (Ankunft: {minutes_to_time(arr)})" for stop, arr in segment["stops"]])
            print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")
    
        print(f"\n🎯 Endstation: {end_stop_name} (Ankunft: {arrival_time_fixed})")
        print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability_fast:.2f}\n")
        last_route = route_id


# Generiere die formattierte Ausgabe





📍 Optimierte zuverlässigste Route von Flughafen Wien Bahnhof nach Seyring Bahnhof:
Backup ab X
Backup ab X
Backup ab X


In [ ]:
for x in range(len(backup_routes)):
    print("Backup ab X")
    
    last_route = None
    grouped_routes = []
    brf = backup_routes[x]
    brf.pop(0)

    for i in range(1, len(backup_routes[x]) - 1, 2):
        
        current_stop, current_time = brf[i - 1]
        route_id, departure_time, arrival_time = brf[i]
        next_stop, _ = brf[i + 1]

        if route_id == last_route:
            grouped_routes[-1]["stops"].append((next_stop, arrival_time))
        else:
            grouped_routes.append({
                "route_id": route_id,
                "start_stop": current_stop,
                "departure_time": departure_time,
                "stops": [(next_stop, arrival_time)]
            })

In [251]:
backup_routes= [[981.0,
  [('Flughafen Wien Bahnhof', 870),
   ('1-S7-W-j24-1', 885.0, 890.0),
   ('Fischamend Bahnhof', 890.0),
   ('1-S7-W-j24-1', 911.0, 915.0),
   ('Flughafen Wien Bahnhof', 915.0),
   ('1-S7-W-j24-1', 919.0, 923.0),
   ('Mannswörth Bahnhof', 923.0),
   ('1-S7-W-j24-1', 923.0, 925.0),
   ('Schwechat Bahnhof', 925.0),
   ('1-S7-W-j24-1', 926.0, 928.0),
   ('Wien Kaiserebersdorf', 928.0),
   ('1-S7-W-j24-1', 929.0, 932.0),
   ('Wien Zentralfriedhof', 932.0),
   ('1-S7-W-j24-1', 932.0, 935.0),
   ('Wien Geiselbergstraße', 935.0),
   ('1-S7-W-j24-1', 935.0, 937.0),
   ('Wien St. Marx', 937.0),
   ('1-S7-W-j24-1', 938.0, 940.0),
   ('Wien Rennweg', 940.0),
   ('1-S2-W-j24-1', 948.0, 950.0),
   ('Wien Mitte-Landstraße', 950.0),
   ('1-S2-W-j24-1', 951.0, 954.0),
   ('Wien Praterstern', 954.0),
   ('1-S2-W-j24-1', 956.0, 959.0),
   ('Wien Traisengasse', 959.0),
   ('1-S2-W-j24-1', 959.0, 960.0),
   ('Wien Handelskai', 960.0),
   ('1-S2-W-j24-1', 961.0, 964.0),
   ('Wien Floridsdorf', 964.0),
   ('1-S2-W-j24-1', 965.0, 968.0),
   ('Wien Siemensstraße', 968.0),
   ('1-S2-W-j24-1', 968.0, 971.0),
   ('Wien Leopoldau', 971.0),
   ('1-S2-W-j24-1', 972.0, 976.0),
   ('Gerasdorf Bahnhof', 976.0),
   ('1-S2-W-j24-1', 976.0, 978.0),
   ('Kapellerfeld Bahnhof', 978.0),
   ('1-S2-W-j24-1', 979.0, 981.0),
   ('Seyring Bahnhof', 981.0)],
  np.float64(0.7452273455163944)],
 [979.0,
  [('Wien Hauptbahnhof', 888.0),
   ('1-S2-W-j24-1', 890.0, 892.0),
   ('Wien Matzleinsdorfer Platz', 892.0),
   ('1-S2-W-j24-1', 893.0, 895.0),
   ('Wien Meidling', 895.0),
   ('1-S2-W-j24-1', 897.0, 900.0),
   ('Wien Hetzendorf', 900.0),
   ('1-S2-W-j24-1', 903.0, 906.0),
   ('Wien Meidling', 906.0),
   ('1-S2-W-j24-1', 908.0, 910.0),
   ('Wien Matzleinsdorfer Platz', 910.0),
   ('1-S2-W-j24-1', 911.0, 913.0),
   ('Wien Hauptbahnhof', 913.0),
   ('1-S2-W-j24-1', 914.0, 915.0),
   ('Wien Quartier Belvedere', 915.0),
   ('1-S2-W-j24-1', 915.0, 918.0),
   ('Wien Rennweg', 918.0),
   ('1-S2-W-j24-1', 918.0, 920.0),
   ('Wien Mitte-Landstraße', 920.0),
   ('1-S2-W-j24-1', 921.0, 924.0),
   ('Wien Praterstern', 924.0),
   ('1-S2-W-j24-1', 926.0, 929.0),
   ('Wien Traisengasse', 929.0),
   ('1-S2-W-j24-1', 929.0, 930.0),
   ('Wien Handelskai', 930.0),
   ('1-S2-W-j24-1', 931.0, 934.0),
   ('Wien Floridsdorf', 934.0),
   ('1-S2-W-j24-1', 935.0, 938.0),
   ('Wien Siemensstraße', 938.0),
   ('1-S2-W-j24-1', 938.0, 941.0),
   ('Wien Leopoldau', 941.0),
   ('1-S2-W-j24-1', 942.0, 946.0),
   ('Gerasdorf Bahnhof', 946.0),
   ('1-S2-W-j24-1', 946.0, 948.0),
   ('Kapellerfeld Bahnhof', 948.0),
   ('1-S2-W-j24-1', 949.0, 951.0),
   ('Seyring Bahnhof', 951.0),
   ('1-S2-W-j24-1', 951.0, 954.0),
   ('Obersdorf Bahnhof', 954.0),
   ('1-S2-W-j24-1', 954.0, 957.0),
   ('Wolkersdorf Bahnhof', 957.0),
   ('1-S2-W-j24-1', 959.0, 961.0),
   ('Ulrichskirchen Bahnhof', 961.0),
   ('1-S2-W-j24-1', 962.0, 964.0),
   ('Schleinbach Bahnhof', 964.0),
   ('1-S2-W-j24-1', 965.0, 968.0),
   ('Ulrichskirchen Bahnhof', 968.0),
   ('1-S2-W-j24-1', 968.0, 971.0),
   ('Wolkersdorf Bahnhof', 971.0),
   ('1-S2-W-j24-1', 973.0, 976.0),
   ('Obersdorf Bahnhof', 976.0),
   ('1-S2-W-j24-1', 976.0, 979.0),
   ('Seyring Bahnhof', 979.0)],
  1.0],
 [942.0,
  [('Wien Praterstern', 907.0),
   ('1-S2-W-j24-1', 917.0, 920.0),
   ('Wien Traisengasse', 920.0),
   ('1-S2-W-j24-1', 920.0, 921.0),
   ('Wien Handelskai', 921.0),
   ('1-S2-W-j24-1', 922.0, 925.0),
   ('Wien Floridsdorf', 925.0),
   ('1-S2-W-j24-1', 926.0, 929.0),
   ('Wien Siemensstraße', 929.0),
   ('1-S2-W-j24-1', 929.0, 932.0),
   ('Wien Leopoldau', 932.0),
   ('1-S2-W-j24-1', 933.0, 937.0),
   ('Gerasdorf Bahnhof', 937.0),
   ('1-S2-W-j24-1', 937.0, 939.0),
   ('Kapellerfeld Bahnhof', 939.0),
   ('1-S2-W-j24-1', 940.0, 942.0),
   ('Seyring Bahnhof', 942.0)],
  1.0]]


In [220]:
import numpy as np